In [1]:
import numpy as np
from liberate import fhe
from liberate.fhe import presets

In [2]:
%%time

# engine = fhe.ckks_engine(devices=[0, 1])
params = presets.params["gold"]
engine = fhe.ckks_engine(verbose=True, **params)

[2023-11-20 02:08:06.916294] I have received the context:


I have received inputs:
        buffer_bit_length		= 62
        scale_bits			= 40
        logN				= 16
        N				= 65,536
        Number of special primes	= 4
        Number of scales		= 34
        Cache folder			= '/home/hanyul/.pyenv/versions/liberate/lib/python3.11/site-packages/liberate/fhe/cache/resources'
        Security bits			= 128
        Quantum security model		= post_quantum
        Security sampling distribution	= uniform
        Number of message bits		= 60
        In total I will be using '1,660' bits out of available maximum '1,661' bits.
        And is it secured?		= True
My RNS primes are [1099510054913, 1099515691009, 1099507695617, 1099516870657, 1099506515969, 1099521458177, 1099503894529, 1099522375681, 1099490000897, 1099523555329, 1099489607681, 1099525128193, 1099486855169, 1099526176769, 1099484889089, 1099529060353, 1099480956929, 1099535220737, 1099469684737, 1099536138241, 1099468767233, 1099537

In [3]:
print("====="*20)
for level in range(engine.num_levels):
    num_of_parties = 5

    amin, amax = -255, 255
    m = engine.example(amin=amin, amax=amax)
    m = np.arange(0, engine.ctx.N//2) + np.arange(0, engine.ctx.N//2) * 1j

    ############################
    ####    generate sks    ####
    ############################
    sks = [engine.create_secret_key() for _ in range(num_of_parties)]

    ############################
    ####    generate pks    ####
    ############################
    pks = [engine.create_public_key(sk=sks[0])]
    crs = engine.clone(pks[0]).data[1]
    for sk in sks[1:]:
        pks.append(engine.multiparty_create_public_key(sk, a=crs))

    cpk = engine.multiparty_create_collective_public_key(pks=pks)

    ###############################
    ####    generate crotks    ####
    ###############################
    delta = np.random.randint(-engine.num_slots*2, engine.num_slots*2-1)
    rotks = [engine.multiparty_create_rotation_key(sk=sks[0], delta=delta)]
    crss = engine.generate_rotation_crs(rotks[0])
    for sk in sks[1:]:
        rotks.append(engine.multiparty_create_rotation_key(sk=sk, delta=delta, a=crss))
    crotk = engine.multiparty_generate_rotation_key(rotks)

    ###################
    ####    enc    ####
    ###################
    ct = engine.encorypt(m, cpk, level=level)


    ############################
    ####    evaluataions    ####
    ############################
    ct = engine.rotate_single(ct, crotk)

    ###################
    ####    dec    ####
    ###################
    pcts = [engine.multiparty_decrypt_head(ct, sks[0])]
    for sk in sks[1:]:
        pcts.append(engine.multiparty_decrypt_partial(ct, sk))
    m_ = engine.multiparty_decrypt_fusion(pcts, level=level)
    m_rolled = np.roll(m, delta)

    print(f"|\t>> abs max error : level  :{level:2d} |\t{engine.absmax_error(m_, m_rolled):.15e}\t| delta : {delta:6d}")
    idx = 5
    for i, (x, y, z) in enumerate(zip(m, m_[:idx], m_rolled[:idx])):
        print(f"|\t{i:3d} | {x.real:18.14f}  | {y.real:18.14f}  | {z.real:18.14f}  |{(y-z).real:18.14f}\t| {level}")
    print("====="*20)

|	>> abs max error : level  : 0 |	3.674358595162630e-10+1.888292899820954e-08j	| delta :  24071
|	  0 |   0.00000000000000  | 8696.99999999992906  | 8697.00000000000000  | -0.00000000007094	| 0
|	  1 |   1.00000000000000  | 8697.99999999994361  | 8698.00000000000000  | -0.00000000005639	| 0
|	  2 |   2.00000000000000  | 8699.00000000003820  | 8699.00000000000000  |  0.00000000003820	| 0
|	  3 |   3.00000000000000  | 8700.00000000002910  | 8700.00000000000000  |  0.00000000002910	| 0
|	  4 |   4.00000000000000  | 8701.00000000005639  | 8701.00000000000000  |  0.00000000005639	| 0
|	>> abs max error : level  : 1 |	3.456079866737127e-10+1.900480128824711e-08j	| delta : -56535
|	  0 |   0.00000000000000  | 23767.00000000005093  | 23767.00000000000000  |  0.00000000005093	| 1
|	  1 |   1.00000000000000  | 23768.00000000013824  | 23768.00000000000000  |  0.00000000013824	| 1
|	  2 |   2.00000000000000  | 23768.99999999995998  | 23769.00000000000000  | -0.00000000004002	| 1
|	  3 |   3.000000